# Generate feed vector

In [1]:
!pip install feedparser

     |████████████████████████████████| 194kB 2.9MB/s 
  Created wheel for feedparser: filename=feedparser-5.2.1-cp36-none-any.whl size=44940 sha256=445d4e85bbc6cddfc8da16da755601e911d691135479ee81f455bf7bbe9b2cd0
  Stored in directory: /root/.cache/pip/wheels/8c/69/b7/f52763c41c5471df57703a0ef718a32a5e81ee35dcf6d4f97f
Successfully built feedparser


In [0]:
import feedparser
import re

In [0]:
# Calculate word frequency in each blog
def get_word_counts(url):
  d = feedparser.parse(url)

  wc = {}
  
  for e in d.entries:
    if 'summary' in e:
      summary = e.summary
    else:
      summary = e.description
    #print('summary: %s' % summary)

    words = get_words(e.title+' '+summary)
    for word in words:
      wc.setdefault(word, 0)
      wc[word]+=1
  
  return d.feed.title, wc

In [0]:
# html-like title and summary -> words
def get_words(html):
  txt = re.compile(r'<[^>]+>').sub('',html)
  words = re.compile(r'[^A-Z^a-z]+').split(txt)
  
  #print('get_words, txt: %s' % txt)
  #print('get_words, words: %s' % words)

  return [word.lower() for word in words if word != '']

In [6]:
# ap_count: number of blog each word appeared
ap_count = {}
word_counts = {}

failed_count = 0

with open('feedlist.txt') as f:
  feed_list = [re.sub(r'\n', '', line) for line in f.readlines()]

for feed_url in feed_list:
  try:
    title, wc = get_word_counts(feed_url)
    word_counts[title] = wc
    for word, count in wc.items():
      ap_count.setdefault(word, 0)
      if count > 1:
        ap_count[word] += 1
  except:
    print('Failed to parse feed %s' % feed_url)
    failed_count += 1

print('Conpleted. Failed to parse %.2f' % (failed_count/len(feed_list)))

Failed to parse feed http://battellemedia.com/index.xml
Failed to parse feed http://feeds.searchenginewatch.com/sewblog
Failed to parse feed http://blog.topix.net/index.rdf
Failed to parse feed http://blogs.abcnews.com/theblotter/index.rdf
Failed to parse feed http://feeds.feedburner.com/ConsumingExperienceFull
Failed to parse feed http://flagrantdisregard.com/index.php/feed/
Failed to parse feed http://featured.gigaom.com/feed/
Failed to parse feed http://gofugyourself.typepad.com/go_fug_yourself/index.rdf
Failed to parse feed http://feeds.feedburner.com/instapundit/main
Failed to parse feed http://jeremy.zawodny.com/blog/rss2.xml
Failed to parse feed http://michellemalkin.com/index.rdf
Failed to parse feed http://beta.blogger.com/feeds/27154654/posts/full?alt=rss
Failed to parse feed http://powerlineblog.com/index.rdf
Failed to parse feed http://feeds.feedburner.com/Publishing20
Failed to parse feed http://scobleizer.wordpress.com/feed/
Failed to parse feed http://www.456bereastreet.

In [0]:
# Calculate document(blog) frequency
word_list = []
for w, bc in ap_count.items():
  ap_prob = float(bc)/len(feed_list)
  if ap_prob > 0.1 and ap_prob < 0.5:
    word_list.append(w)

In [0]:
# Create word list for each blog
with open('blogdata.txt', 'w') as f:
  f.write('Blog')
  for word in word_list:
    f.write('\t%s' % word)
  f.write('\n')
  for blog, wc in word_counts.items():
    f.write(blog)
    for word in word_list:
      if word in wc:
        f.write('\t%s' % wc[word])
      else:
        f.write('\t0')
    f.write('\n')


# Clusters: Binary

In [0]:
def readfile(filename):
  with open(filename) as f:
    lines = f.readlines()
  
  col_names = lines[0].strip().split('\t')[1:]
  row_names = []
  data = []
  for line in lines[1:]:
    p = line.strip().split('\t')
    row_names.append(p[0])
    data.append([float(x) for x in p[1:]])
  
  return row_names, col_names, data

In [0]:
# Define the distance
from scipy.spatial.distance import correlation
def pearson(x, y):
  return correlation(x, y)

In [0]:
# Define the binary-cluster
class bicluster:
  def __init__(self, vec, left=None, right=None, distance=0.0, id=None):
    self.left = left
    self.right = right
    self.vec = vec
    self.id = id
    self.distance = distance

In [0]:
# Clustering: search top-2 most similar classes
def hcluster(rows, distance=pearson):
  distances = {}
  current_clust_id = -1

  clust = [bicluster(rows[i], id=i) for i in range(len(rows))]

  while len(clust) > 1:
    lowest_pair = (0, 1)
    closest = distance(clust[0].vec, clust[1].vec)

    for i in range(len(clust)):
      for j in range(i+1, len(clust)):
        if (clust[i].id, clust[j].id) not in distances:
          distances[(clust[i].id, clust[j].id)] = distance(clust[i].vec, clust[j].vec)

        d = distances[(clust[i].id, clust[j].id)]
        #print(d)

        if d < closest:
          closest = d
          lowest_pair = (i, j)
    
    # Create new cluster by merging 2 clusters from lowest pair
    ## calculate average to merge 2 clusters
    merge_vec = [(clust[lowest_pair[0]].vec[i] + clust[lowest_pair[1]].vec[i])/2.0 for i in range(len(clust[0].vec))]
    ## make new cluster
    new_cluster = bicluster(merge_vec, left=clust[lowest_pair[0]], right=clust[lowest_pair[1]], distance=closest, id=current_clust_id)

    # id of new clusters is minus value
    current_clust_id -= 1
    del clust[lowest_pair[1]]
    del clust[lowest_pair[0]]
    clust.append(new_cluster)

  return clust[0]


In [0]:
# Visualization
def print_clust(clust, labels=None, n=0):
  for i in range(n):
    print(' ',end='')
  if clust.id < 0:
    print('-')
  else:
    if labels == None:
      print(clust.id)
    else:
      print(labels[clust.id])
  
  if clust.left != None:
    print_clust(clust.left, labels=labels, n=n+1)
  if clust.right != None:
    print_clust(clust.right, labels=labels, n=n+1)

In [35]:
blognames, words, data = readfile('blogdata.txt')
clust = hcluster(data)
#print(clust)
print_clust(clust, labels=blognames)

-
 PaulStamatiou.com - Technology, Design and Photography
 -
  Autoblog
  -
   O'Reilly Radar
   -
    TechEBlog
    -
     pharyngula
     -
      -
       -
        Mashable
        -
         Engadget RSS Feed
         Wired
       -
        TechCrunch
        -
         Schneier on Security
         -
          -
           -
            The Official Google Blog
            -
             Google Blogoscoped
             -
              Google Operating System
              Search Engine Roundtable
           -
            Seth's Blog
            -
             Lifehack - Feed
             ShoeMoney
          -
           -
            TMZ.com
            -
             Captain's Quarters
             -
              -
               NB Blog Feed
               ThinkProgress
              -
               Latest from Crooks and Liars
               -
                Boing Boing
                -
                 Slashdot
                 Techdirt.
           -
            blog maver

# Clusters: k-means

In [0]:
import random

In [0]:
def kcluster(rows, distance=pearson, k=4):
  ranges = [(min(row[i] for row in rows), max(row[i] for row in rows)) for i in range(len(rows[0]))]
  # randomly setting k-clusters at first. each value is range(min, max)
  clusters = [[random.random()*(ranges[i][1]-ranges[i][0])+ranges[i][0] for i in range(len(rows[0]))] for j in range(k)]
  
  last_matches = None
  
  for t in range(100):
    print('iteration %d' % t)
    best_matches = [[] for j in range(k)]

    for i in range(len(rows)):
      row = rows[i]
      best_match = None
      for j in range(k):
        d = distance(clusters[j], row)
        if best_match == None or d < distance(clusters[best_match], row):
          best_match = j
      best_matches[best_match].append(i)
    
    # matches time t == matches time t-1 -> end
    if best_matches == last_matches:
      break
    last_matches = best_matches

    # reset k-positions
    for i in range(k):
      avgs = [0.0]*len(rows[0])
      if len(best_matches[i]) > 0:
        for row_id in best_matches[i]:
          for m in range(len(rows[row_id])):
            avgs[m] += rows[row_id][m]
        for j in range(len(avgs)):
          avgs[j] /= len(best_matches[i])
      clusters[i] = avgs
  
  return best_matches


In [51]:
blognames, words, data = readfile('blogdata.txt')
kclust = kcluster(data, k=10)

iteration 0
iteration 1
iteration 2


/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


iteration 3
iteration 4


In [55]:
for i in range(10):
  print([blognames[r] for r in kclust[i]])

['The Viral Garden', 'WIL WHEATON dot NET', 'BuzzMachine', 'Matt Cutts: Gadgets, Google, and SEO', 'mezzoblue', "Neil Gaiman's Journal", 'Oilman', 'Derek Powazek', 'Steve Pavlina']
['PaulStamatiou.com - Technology, Design and Photography', "Seth's Blog", 'Lifehack - Feed', 'ShoeMoney']
[]
['Guy Kawasaki', 'Copyblogger', 'ProBlogger']
['Signal v. Noise', 'Eschaton', 'Creating Passionate Users', "Joi Ito's Web", 'pharyngula', 'The Dish', '43 Folders', 'blog maverick', 'kottke.org', 'Schneier on Security', 'ongoing by Tim Bray']
['Gizmodo', 'Mashable', 'The Write News', 'Deadspin', 'Engadget RSS Feed', 'Gothamist', 'Joel on Software', 'Kotaku', 'Lifehacker', 'Quick Online Tips', 'Wired']
['NB Blog Feed', 'ThinkProgress']
["O'Reilly Radar", 'Slashdot', 'Autoblog', 'Boing Boing', "Captain's Quarters", 'Latest from Crooks and Liars', 'TechCrunch', 'Techdirt.', 'TechEBlog', 'TMZ.com']
['Google Blogoscoped', 'Google Operating System', 'Search Engine Roundtable']
['The Official Google Blog']


# More

## Distance: Tanimoto/Jaccard
 - if the data is binary(0 or 1, exist or none), Tanimoto/Jaccard is useful to calculate a **Duplication rate**

In [0]:
def tanimoto(v1, v2):
  c1, c2, shr = 0, 0, 0
  for i in range(len(v1)):
    if v1[i] != 0:
      c1 += 1
    if v2[i] != 0:
      c2 += 1
    if v1[i] != 0 and v2[i] != 0:
      shr += 1
  
  return 1.0 - (float(shr)/(c1+c2-shr))

## Plot: into 2 dimensions

In [0]:
import numpy as np
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt

In [91]:
!pip install Pillow

In [0]:
from PIL import Image, ImageDraw

In [0]:
def data2numpy(data):
  return np.array(data)

In [0]:
def svd(array, dim=2):
  U, S, V = svds(array, k=dim)
  S = np.diag(S)
  return np.dot(U, S)

In [0]:
def draw2d(data, labels, dim=2):
  array = data2numpy(data)
  array_2d = svd(array, dim)
  
  x = [a[0] for a in array_2d]
  y = [a[1] for a in array_2d]
  
  img = Image.new('RGB', (2000, 2000), (255, 255, 255))
  draw = ImageDraw.Draw(img)
  for i in range(len(labels)):
    draw.text((x[i]*10,y[i]*10), labels[i], (0,0,0))
  
  img.save('blog.jpg', 'JPEG')
  
  """
  #plt.scatter(x, y)
  for i in range(len(labels)):
    plt.text(x[i], y[i], labels[i])
  plt.savefig('blog.png')
  """
  return

In [0]:
draw2d(data, blognames, dim=2)